In [1]:
from config import *
from constants import *
import xarray as xr
from COSIPY import prereq_res, construct_resampled_ds
from cosipy.modules.evaluation import * 
import pandas as pd
import numpy as np
import sys


/home/niki/anaconda3/envs/cosipy_test/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# Load sample file
result = xr.open_dataset("/home/niki/Dokumente/cosipy_test_space/data/output/"+
                         "HEF_COSMO_300m_1999_2010_DREAM_19990101-20100101_num9.nc")

result

<xarray.Dataset>
Dimensions:       (lat: 14, lon: 30, time: 96444)
Coordinates:
  * time          (time) datetime64[ns] 1999-01-01 1999-01-01 ... 2010-01-01
  * lat           (lat) float64 46.79 46.79 46.79 46.79 ... 46.82 46.82 46.82
  * lon           (lon) float64 10.72 10.73 10.73 10.73 ... 10.8 10.8 10.8 10.8
Data variables: (12/37)
    HGT           (lat, lon) float32 ...
    MASK          (lat, lon) float32 ...
    SLOPE         (lat, lon) float32 ...
    ASPECT        (lat, lon) float32 ...
    T2            (time, lat, lon) float64 ...
    RH2           (time, lat, lon) float64 ...
    ...            ...
    CONDENSATION  (time, lat, lon) float64 ...
    DEPOSITION    (time, lat, lon) float64 ...
    REFREEZE      (time, lat, lon) float64 ...
    subM          (time, lat, lon) float64 ...
    Z0            (time, lat, lon) float64 ...
    surfM         (time, lat, lon) float64 ...
Attributes: (12/63)
    Start_from_restart_file:                    False
    Stake_evaluation:                           False
    WRF_simulation:                             False
    Compression_level:                          2
    Slurm_use:                                  True
    Full_fiels:                                 False
    ...                                         ...
    water_density:                              1000.0
    ice_density:                                917.0
    air_density:                                1.1
    sigma:                                      5.67e-08
    zero_temperature:                           273.16
    Surface_emission_coeff:                     0.99

In [25]:
## Start resampling by hand routine

tsla_observations = pd.read_csv(tsl_data_file)
tsla_observations

dates, clean_day_vals, secs, holder = prereq_res(result.sel(time=slice("2000-01-01","2009-12-31")))
print(dates, clean_day_vals, secs, holder)

resampled_array = resample_by_hand(holder, result.sel(time=slice("2000-01-01","2009-12-31")).SNOWHEIGHT.values, secs, clean_day_vals)
resampled_out = construct_resampled_ds(result.sel(time=slice("2000-01-01","2009-12-31")), resampled_array, dates.values)
resampled_out['HGT'] = (('lat','lon'), result['HGT'].data)
resampled_out['MASK'] = (('lat','lon'), result['MASK'].data)

tsl_out = create_tsl_df(resampled_out, min_snowheight, tsl_method, tsl_normalize)

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2009-12-22', '2009-12-23', '2009-12-24', '2009-12-25',
               '2009-12-26', '2009-12-27', '2009-12-28', '2009-12-29',
               '2009-12-30', '2009-12-31'],
              dtype='datetime64[ns]', length=3653, freq=None) [ 946684800000000000  946771200000000000  946857600000000000 ...
 1262044800000000000 1262131200000000000 1262217600000000000] [ 946684800000000000  946684800000000000  946688400000000000 ...
 1262293200000000000 1262296800000000000 1262300400000000000] [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0

In [73]:
tsl_out.loc[tsl_out['time'].isin(tsla_observations['LS_DATE'])]

,time,Med_TSL,Mean_TSL,Std_TSL,Max_TSL,Min_TSL,Flag
26,2000-01-27,0.000000,0.000000,-2.013260,0.000000,0.000000,1.0
58,2000-02-28,0.000000,0.000000,-2.013260,0.000000,0.000000,1.0
170,2000-06-19,0.000000,0.000000,-2.013260,0.000000,0.000000,1.0
186,2000-07-05,0.058158,0.058158,-1.998620,0.072798,0.043518,0.0
266,2000-09-23,0.555630,0.555630,-1.956499,0.612391,0.498869,0.0
...,...,...,...,...,...,...,...
3482,2009-07-14,0.044016,0.044016,-2.012763,0.044514,0.043518,0.0
3498,2009-07-30,0.297253,0.297253,-2.003701,0.306813,0.287693,0.0
3514,2009-08-15,0.482437,0.482437,-1.936264,0.559432,0.405441,0.0
3530,2009-08-31,0.613547,0.613547,-1.943485,0.683323,0.543771,0.0


In [74]:
tsla_observations.iloc[6:None]

,LS_DATE,CC_area,CC_total_port,DC_area,DIC_area,DIC_max,IC_area,LS_ID,LS_SAT,RGI_ID,...,class_coverage,glacier_DEM_max,glacier_DEM_min,glacier_area,tool_version,scene,TSL_stddev_norm,TSL_max_norm,TSL_min_norm,TSL_normalized
6,2000-01-27,0.128650,2.635219,0.000000,0.771875,3681.0,0.771875,LT05_L1TP_193027_20000127_20171212_01_T1,LANDSAT_5,RGI60-11.00897,...,60.910861,3698,2412,8.014875,0.8.1,193027,-1.860917,0.054432,0.001555,0.031882
7,2000-02-28,0.027090,0.371488,0.000000,0.128655,2926.0,0.128655,LT05_L1TP_193027_20000228_20171211_01_T1,LANDSAT_5,RGI60-11.00897,...,90.986062,3698,2412,8.014875,0.8.1,193027,-1.854551,0.080093,0.005443,0.044323
8,2000-06-19,0.291125,3.106597,0.331701,2.193678,3682.0,1.861979,LT05_L1TP_193027_20000619_20171211_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.865007,0.210731,0.171073,0.195956
9,2000-07-05,0.390600,4.593407,0.269377,2.472011,3682.0,2.202634,LT05_L1TP_193027_20000705_20171210_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.862746,0.293157,0.235614,0.280715
10,2000-09-23,0.051182,0.696388,0.004490,0.550438,3651.0,0.545948,LT05_L1TP_193027_20000923_20171211_01_T1,LANDSAT_5,RGI60-11.00897,...,91.700154,3698,2412,8.014875,0.8.1,193027,-1.851573,0.087092,0.000000,0.055210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2009-07-14,1.659006,17.703254,0.595720,2.985878,3682.0,2.390158,LT05_L1TP_193027_20090714_20161023_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.870814,0.255832,0.237170,0.248834
64,2009-07-30,3.961122,42.794823,0.751441,3.690090,3681.0,2.938649,LT05_L1TP_193027_20090730_20161027_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.869126,0.314930,0.291602,0.309487
65,2009-08-15,1.184912,13.088718,1.218616,4.678650,3649.0,3.460037,LT05_L1TP_193027_20090815_20161022_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.865531,0.380249,0.342924,0.361586
66,2009-08-31,1.272939,14.641502,1.557160,5.734950,3696.0,4.177785,LT05_L1TP_193027_20090831_20161021_01_T1,LANDSAT_5,RGI60-11.00897,...,100.000000,3698,2412,8.014875,0.8.1,193027,-1.830984,0.486003,0.372473,0.481337


In [40]:
## other version of resampling
a_resampled_out = resample_output(result.sel(time=slice("2000-01-01", "2019-12-31"))).isel(time=slice(None, -1)) # for some reason it features 2010-01-01 
#a_tsl_out = create_tsl_df(a_resampled_out, min_snowheight, tsl_method, tsl_normalize)

## xarray version of resampling
def resample_xarray(cos_output):
    ds = cos_output
    ds = ds[['SNOWHEIGHT','HGT','MASK']]
    ds_daily = ds.resample(time="1d", keep_attrs=True).mean(dim='time')
    return ds_daily

test = resample_xarray(result.sel(time=slice("2000-01-01", "2019-12-31"))).isel(time=slice(None, -1))
test


Required time for resample only:  0:00:11.498052


/home/niki/anaconda3/envs/cosipy_test/lib/python3.10/site-packages/xarray/core/common.py:1127: UserWarning: Passing ``keep_attrs`` to ``resample`` has no effect and will raise an error in xarray 0.20. Pass ``keep_attrs`` directly to the applied function, e.g. ``resample(...).mean(keep_attrs=True)``.
  warnings.warn(


<xarray.Dataset>
Dimensions:     (lat: 14, lon: 30, time: 3653)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2009-12-31
  * lat         (lat) float64 46.79 46.79 46.79 46.79 ... 46.82 46.82 46.82
  * lon         (lon) float64 10.72 10.73 10.73 10.73 ... 10.8 10.8 10.8 10.8
Data variables:
    SNOWHEIGHT  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    HGT         (time, lat, lon) float32 3.227e+03 3.244e+03 ... 2.439e+03
    MASK        (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [41]:
## Start testing if resampling worked
np.testing.assert_allclose(a_resampled_out.SNOWHEIGHT.values, test.SNOWHEIGHT.values, err_msg="These two are not equal")
#these two are equivalent 

In [42]:
np.testing.assert_allclose(a_resampled_out.SNOWHEIGHT.values, resampled_out.SNOWHEIGHT.values, err_msg="These two are not equal")
#there seems to be a difference between the two versions - date time 

In [44]:
checkup = ~np.isclose(a_resampled_out.SNOWHEIGHT.values, resampled_out.SNOWHEIGHT.values, rtol=1e-4, equal_nan=True)
checkup


#get indices where it is false
indices = np.where(checkup)
indices

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [49]:
## Resampling led to same values, now check the next routine which is creating the TSL data
a_resampled_out

<xarray.Dataset>
Dimensions:     (lat: 14, lon: 30, time: 3653)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2009-12-31
  * lat         (lat) float64 46.79 46.79 46.79 46.79 ... 46.82 46.82 46.82
  * lon         (lon) float64 10.72 10.73 10.73 10.73 ... 10.8 10.8 10.8 10.8
Data variables:
    SNOWHEIGHT  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    HGT         (time, lat, lon) float32 3.227e+03 3.244e+03 ... 2.439e+03
    MASK        (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [75]:
try:
    a_tsl_out = create_tsl_df(a_resampled_out, min_snowheight, tsl_method, tsl_normalize)
except:
    print("Numba issue exists.")

Calculating TSLA using conservative. Normalization is set to True.
Max elev. 3656.1287 .
 Min elev. 2442.7822
Numba issue exists.


In [57]:
#numba causes issues here, for whatever a reason. instead just use a copy of the function without numba
def tsl_method_conservative_nojit(snowheights, hgts, mask, min_snowheight):
    amed = np.zeros(snowheights.shape[0])
    amean = np.zeros(snowheights.shape[0])
    astd = np.zeros(snowheights.shape[0])
    amax = np.zeros(snowheights.shape[0])
    amin = np.zeros(snowheights.shape[0])
    flag = np.zeros(snowheights.shape[0])

    for n in np.arange(0, snowheights.shape[0]):
        #min altitude of snow
        #print("Processing ", n, "out of ", snowheights.shape[0])
        filtered_elev_snow = np.nanmin(np.where(snowheights[n,:,:]>=min_snowheight, hgts, np.nan).ravel())
        #print(filtered_elev_snow)
        #this line is basically redudant, ensures that snowline altitude cannot fall below glacier altitude
        filtered_elev_snow = np.nanmax(np.append(filtered_elev_snow, np.nanmin(np.where(mask==1, hgts, np.nan)))) #numba does not support np.maximum
        #print(filtered_elev_snow)
        #now for snow-free surfaces
        filtered_elev_nosnow = np.nanmax(np.where(snowheights[n,:,:]<min_snowheight, hgts, np.nan))
        #print(filtered_elev_nosnow)
        if np.isnan(filtered_elev_nosnow):
            #print("Glacier seems to be fully snow-covered. Assigning minimum elevation.")
            filtered_elev_nosnow = np.nanmin(np.where(mask==1, hgts, np.nan))
            #print(filtered_elev_nosnow,"m a.s.l.")
            flag[n] = 1
        else:
            #technically also redudant, ascertain that max. elevation of glacier is not exceeded
            filtered_elev_nosnow = np.nanmin(np.append(filtered_elev_nosnow, np.nanmax(np.where(mask==1, hgts, np.nan)))) #numba does not support minimum/maximum of numpy
        amed[n] = np.nanmedian(np.append(filtered_elev_snow, filtered_elev_nosnow))
        amean[n] = np.nanmean(np.append(filtered_elev_snow, filtered_elev_nosnow))
        astd[n] = np.nanstd(np.append(filtered_elev_snow, filtered_elev_nosnow)) #might be a bit large.. std of all values essentially function of resolution  
        amax[n] = np.nanmax(np.append(filtered_elev_snow, filtered_elev_nosnow))
        amin[n] = np.nanmin(np.append(filtered_elev_snow, filtered_elev_nosnow))

    return (amed, amean, astd, amax, amin, flag)

def calculate_tsl_byhand_nojit(snowheights,hgts,mask, min_snowheight,tsl_method, tsl_normalize):
    max_elev = np.nanmax(np.where(mask==1, hgts, np.nan))
    min_elev = np.nanmin(np.where(mask==1, hgts, np.nan))
    print("Calculating TSLA using {}. Normalization is set to {}.".format(tsl_method, tsl_normalize))
    print("Max elev.", max_elev,".\n Min elev.", min_elev)
    if tsl_method == 'mantra':
        amed, amean, astd, amax, amin, flag = tsl_method_mantra(snowheights, hgts, mask, min_snowheight)
    elif tsl_method == 'conservative':
        amed, amean, astd, amax, amin, flag = tsl_method_conservative_nojit(snowheights, hgts, mask, min_snowheight)
    else:
        amed, amean, astd, amax, amin, flag = tsl_method_gridsearch(snowheights, hgts, mask, min_snowheight)
    if tsl_normalize:
        #Start normalizing:
        amed_norm = (amed - min_elev) / (max_elev - min_elev)
        amean_norm = (amean - min_elev) / (max_elev - min_elev)
        astd_norm = (astd - min_elev) / (max_elev - min_elev)
        amax_norm = (amax - min_elev) / (max_elev - min_elev)
        amin_norm = (amin - min_elev) / (max_elev - min_elev)
        return (amed_norm, amean_norm, astd_norm, amax_norm, amin_norm, flag)
    else:
        return (amed, amean, astd, amax, amin, flag)

def create_tsl_df_nojit(cos_output,min_snowheight, tsl_method, tsl_normalize):
    times = datetime.now()
    amed,amean,astd,amax,amin,flag = calculate_tsl_byhand_nojit(cos_output.SNOWHEIGHT.values, cos_output.HGT.values, cos_output.MASK.values, min_snowheight, tsl_method, tsl_normalize)
    print(amed)
    tsl_df = pd.DataFrame({'time':pd.to_datetime(cos_output.time.values),
                            'Med_TSL':amed,
                            'Mean_TSL':amean,
                            'Std_TSL':astd,
                            'Max_TSL':amax,
                            'Min_TSL':amin,
                            'Flag': flag})
    print("Time required for calculating TSL only :", datetime.now()-times)
    return tsl_df

In [58]:
a_tsl_out = create_tsl_df_nojit(a_resampled_out, min_snowheight, tsl_method, tsl_normalize)

Calculating TSLA using conservative. Normalization is set to True.
Max elev. 3656.1287 .
 Min elev. 2442.7822


/tmp/ipykernel_19149/3061086146.py:19: RuntimeWarning: All-NaN slice encountered
  filtered_elev_nosnow = np.nanmax(np.where(snowheights[n,:,:]<min_snowheight, hgts, np.nan))


[0. 0. 0. ... 0. 0. 0.]
Time required for calculating TSL only : 0:00:43.580121


In [59]:
a_tsl_out

,time,Med_TSL,Mean_TSL,Std_TSL,Max_TSL,Min_TSL,Flag
0,2000-01-01,0.0,0.0,-2.01326,0.0,0.0,1.0
1,2000-01-02,0.0,0.0,-2.01326,0.0,0.0,1.0
2,2000-01-03,0.0,0.0,-2.01326,0.0,0.0,1.0
3,2000-01-04,0.0,0.0,-2.01326,0.0,0.0,1.0
4,2000-01-05,0.0,0.0,-2.01326,0.0,0.0,1.0
...,...,...,...,...,...,...,...
3648,2009-12-27,0.0,0.0,-2.01326,0.0,0.0,1.0
3649,2009-12-28,0.0,0.0,-2.01326,0.0,0.0,1.0
3650,2009-12-29,0.0,0.0,-2.01326,0.0,0.0,1.0
3651,2009-12-30,0.0,0.0,-2.01326,0.0,0.0,1.0


In [60]:
tsl_out

,time,Med_TSL,Mean_TSL,Std_TSL,Max_TSL,Min_TSL,Flag
0,2000-01-01,0.0,0.0,-2.01326,0.0,0.0,1.0
1,2000-01-02,0.0,0.0,-2.01326,0.0,0.0,1.0
2,2000-01-03,0.0,0.0,-2.01326,0.0,0.0,1.0
3,2000-01-04,0.0,0.0,-2.01326,0.0,0.0,1.0
4,2000-01-05,0.0,0.0,-2.01326,0.0,0.0,1.0
...,...,...,...,...,...,...,...
3648,2009-12-27,0.0,0.0,-2.01326,0.0,0.0,1.0
3649,2009-12-28,0.0,0.0,-2.01326,0.0,0.0,1.0
3650,2009-12-29,0.0,0.0,-2.01326,0.0,0.0,1.0
3651,2009-12-30,0.0,0.0,-2.01326,0.0,0.0,1.0


In [61]:
np.allclose(a_tsl_out['Med_TSL'], tsl_out['Med_TSL'])


True

In [64]:
for column in tsl_out.columns:
    if column != "time":
        print("Column ", column, "are equal?")
        print(np.allclose(tsl_out[column],a_tsl_out[column]))

Column  Med_TSL are equal?
True
Column  Mean_TSL are equal?
True
Column  Std_TSL are equal?
True
Column  Max_TSL are equal?
True
Column  Min_TSL are equal?
True
Column  Flag are equal?
True
